In [1]:
from IPython.display import display, Image
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
Image('gan01')

进化流程：
- 第一代的Generator，输入随机变量，产生一些图片，第一代的Discriminator为二元分类器，判断输入的是Generator生成的图片，还是真实图片；
- 根据DiscriminatorV1产生GeneratorV2，它生成的图片能骗过DiscriminatorV1，使它认为是真实图片，然后有了DiscriminatorV2，它能分辨是GeneratorV2生成的图片还是真实图片；
- 以此类推，产生GeneratorV3和DiscriminatorV3；

#### GAN - Discriminator训练方法：
- Discriminator为二元分类器，真实图片为1，Generator生成的图片为0。它是神经网络结构，网络层数和单元数自己决定的。Generator是输入随机变量，会生成一些图片；
- 首先初始化GeneratorV1，参数都是随机产生的，然后输入一组vector，生成假的图片，将它们标记为0；
- 在从数据集里跳出一组真的图片，为1。将标记为0的假图片和标记为1的真图片放入DiscriminatorV1里，进行训练，使它能辨别出是GeneratorV1生成的假图片还是真图片；

In [ ]:
Image('gan02')

#### GAN - Generator训练方法：
- 首先Generator和Discriminator组成一个神经网络，Generator输出的图片是某层的隐藏层输出。前面几层为Generator，后面几层为Discriminator，中间有一层输出图片。
- 固定DiscriminatorV1，使用梯度下降，更新Generator的参数，产生GeneratorV2，使得生成的图片能骗过DiscriminatorV1，即值越接近1越好。

In [ ]:
Image('gan03')

### GAN的基本理念

#### 最大似然估计
- 给定一个数据分布$P_{data}(x)$，x为一张图片，把图片上的像素展开为很长的一组向量。data为多张图片，即多组向量，集合起来描述一个分布；
- 找一个分布$P_{G}(x; θ)$，参数θ控制决定$P_{G}(x; θ)$的分布。$P_{data}(x)$是已有数据集的分布，找到一组参数θ让$P_{G}(x; θ)$的分布跟$P_{data}(x)$的分布越接近越好；
- 使用神经网络可以生成非常复杂的分布$P_{G}(x; θ)$，即$P_{G}(x; θ)$为神经网络，θ为神经网络的参数；初始化一个神经网络，为函数G，输入为向量z。通常z为低维向量，比如输出的图片为$96 \times 96$，输入的向量为$32 \times 32$或$64 \times 64$等低维向量。将低维向量z放入神经网络G里输出一张图片x，即$G(z)=x$。z为一个高斯分布（或其它分布）的例子，通过神经网络G转换为另一个复杂分布x。现在就是训练神经网络找一组参数θ，使得神经网络产生的分布$P_{G}(x; θ)$跟$P_{data}(x)$的分布越接近越好。

In [ ]:
Image('gan05')

- Generator G，G为函数（一般为神经网络），输入为z，输出为x，即$G(z)=x$。给定一个先验分布$P_{prior}(z)$，函数G定义一个概率分布$P_{G}(x)$;
- Discriminator D，D为函数，输入为x，输出为标量（判断与真实图片的差异性，值越接近0，差异性越大，相反则越小）。可以估计$P_{G}(x)$分布和$P_{data}(x)$分布的差异性；
- 两个分布的差异性，通过计算函数$V(G, D)$：$G^{*}=arg \ \underset{G}{\text {min}} \  \underset{D}{\text {max}} \ V(G, D)$，找一个D让$V(G, D)$最大化，再找一个G去最小化$\underset{D}{\text {max}} \ V(G, D)$

公式$\underset{D}{\text {max}}$，找一个D去最大化$V(G, D)$，G为Generator，假设只有三个$(G_{1}, G_{2}, G_{3})$，如下图（实际上Generator由神经网络定义，网络参数不同，Generator就不同，所以有无穷多个）：

In [ ]:
Image('gan06')

- $V(G, D)$为函数，固定$G_{1}$，改变Discriminator D的参数会获得不同的数值（Discriminator为神经网络，没法用一个维度来表示参数。上图为讲解方便，使用一个维度来表示Discriminator的参数）；
- 固定$G_{1}$，给不同的改变Discriminator会得到不同的$V(G_{1}, D)$，同理可得$V(G_{2}, D), V(G_{3}, D)$；
- 假设G为$G_{1}$的话，$\underset{D}{\text {max}} \ V(G, D)$即上图$G_{1}$中的红色点的值为12，假设G为$G_{2}$，$\underset{D}{\text {max}} \ V(G, D)$即上图$G_{2}$中的红色点的值为14，G为$G_{3}$为5；

现在要找一个G可以最小化$\underset{D}{\text {max}} \ V(G, D)$，已知固定$G_{1}$，$\underset{D}{\text {max}} \ V(G_{1}, D)$为12，固定$G_{2}$为14，固定$G_{3}$为5。因为$G_{3}$能最小化$\underset{D}{\text {max}} \ V(G, D)$，所以$G^{*}$=$G_{3}$。$D^{*}$则是让$V(G, D)$最大化的，即上图中蓝色线上的绿色点。

$V$的公式，表示给定一个Generator，最大化$V(G, D)$就是估计$P_{G}$和$P_{data}$间的差异（以上图中的$G_{1}$为例，红色点到横坐标轴的高度即为$P_{G}$和$P_{data}$的差异，所以$G_{3}$差异最小。只要找到一个G，能让高度（差异）最小即可）：
$$V=E_{x\sim P_{data}}[\log D(x)] + E_{x\sim P_{G}}[\log (1 - D(x))]$$

- 假设给定$G$，如何选取$D$使其最大化，即$\underset{D}{\text {max}} \ V(G, D)$，$V$如上式，表示从分布$P_{data}$里抽取一个$x$，取$\log D(x)$的期望值，D为Discriminator。加上从分布$P_{G}$里抽取一个$x$，取$\log (1 - D(x))$的期望值。将$V$转为积分形式：
$$V=E_{x\sim P_{data}}[\log D(x)] + E_{x\sim P_{G}}[\log (1 - D(x))]\\
=\underset{x}{\int } P_{data}(x) \log D(x) dx + \underset{x}{\int } P_{G}(x) \log (1-D(x)) dx \\
=\underset{x}{\int } [P_{data}(x) \log D(x)  +  P_{G}(x) \log (1-D(x))] dx$$
- 假设$D(x)$可以为任何值，给定x，找到一个$D^{*}$使其$V$最大化，即中括号里的每一项都分开考虑，对每一个$x$使下式最大化即可：
$$P_{data}(x) \log D(x)  +  P_{G}(x) \log (1-D(x))$$
data是给定的，所以$P_{data}(x)$就是一个标量值设为$a$，假设$G$是给定的，则$P_{G}(x)$也是一个标量值设为$b$；
- 上式转化为：$f(D)=a \log D(x) + b \log (1-D(x))$，找到一个$D$使其最大化，即先对其微分，求出极值：
$$\frac{df(D)}{dD}=a \times \frac{1}{D} + b \times \frac{1}{1-D} \times (-1)=0\\
a \times \frac{1}{D^{*}} = b \times \frac{1}{1-D^{*}} \\
a (1-D^{*}) = b \times (D^{*}) \\
a - a D^{*} = b D^{*}\\
D^{*} = \frac{a}{a+b}=\frac{P_{data}(x)}{P_{data}(x)+P_{G}(x)}$$

在将$G_{1}$代入上式，如下图。接着将$G_{1}$和$D_{1}^{*}$代入公式$V$中，即可求出高度（两个分布的差异）。

In [ ]:
Image('gan07')

#### 为什么公式$V$能衡量两个分布的差异性？
- 已知$D^{*} =\frac{P_{data}(x)}{P_{data}(x)+P_{G}(x)}$，在给定$G$的情况下，$\underset{D}{\text {max}} V(G, D) = V(G, D^{*})$，将$D^{*}$代入式子$V$中：
$$V=E_{x\sim P_{data}}[\log D(x)] + E_{x\sim P_{G}}[\log (1 - D(x))]\\
=E_{x\sim P_{data}}[\log \frac {P_{data}(x)}{P_{data}(x)+P_{G}(x)}] + E_{x\sim P_{G}}[\log \frac {P_{G}(x)}{P_{data}(x)+P_{G}(x)}]\\
=\underset {x}{\int } P_{data}(x) \log \frac {P_{data}(x)}{P_{data}(x)+P_{G}(x)} dx + \underset {x}{\int } P_{G}(x) \log \frac {P_{G}(x)}{P_{data}(x)+P_{G}(x)} dx\\
=\underset {x}{\int } P_{data}(x) \log \frac {\frac{1}{2}P_{data}(x)}{(P_{data}(x)+P_{G}(x))/2} dx + \underset{x}{\int } P_{G}(x) \log \frac {\frac{1}{2} P_{G}(x)}{(P_{data}(x)+P_{G}(x))/2} dx\\
=2\log \frac{1}{2} + \underset {x}{\int } P_{data}(x) \log \frac {P_{data}(x)}{(P_{data}(x)+P_{G}(x))/2} dx + \underset{x}{\int } P_{G}(x) \log \frac {P_{G}(x)}{(P_{data}(x)+P_{G}(x))/2} dx\\
=-2\log2 + \text {KL}(P_{data}(x)\parallel \frac{P_{data}(x)+P_{G}(x)}{2}) + \text {KL}(P_{G}(x)\parallel \frac{P_{data}(x)+P_{G}(x)}{2})$$

上式中后两项为$P_{data}(x)$和$P_{data}(x)$及$P_{G(x)}$平均的$KL$分布，$P_{G}(x)$和$P_{data}(x)$及$P_{G(x)}$平均的$KL$分布。数学上有个Jensen-Shannon divergence：
$$\text {JSD}(P \parallel Q) = \frac {1}{2}D (P \parallel M) + \frac {1}{2} D(Q \parallel  M)\\
M = \frac {1}{2} (P+Q)$$
于是上式可写为：
$$-2 \log 2 + 2 \text {JSD}(P_{data}(x) \parallel P_{G}(x)$$
$JSD$的最大值为$\log 2$，当两个分布完全没有交集时，两个分布完全一致时，值为0。

总结一下：
- 给定Generator G，Discriminator D；
- 通过以下公式找到一个最佳的$G^{*}$：$$G^{*}=arg \ \underset{G}{\text {min}} \ \underset{D}{\text {max}} \ V(G, D)$$
其中：
$$V=E_{x\sim P_{data}}[\log D(x)] + E_{x\sim P_{G}}[\log (1 - D(x))]$$
- 给定G，找到一个$D$最大化$V(G, D)$，$\underset{D}{\text {max}} \ V(G, D)$：
$$=-2 \log 2 + 2 \text {JSD}(P_{data}(x) \parallel P_{G}(x)$$
- 找一个G，可以让式子$\underset{D}{\text {max}} \ V(G, D)$最小化，当两个分布完全一致时，$JSD$为0，这时$\underset{D}{\text {max}} \ V(G, D)$达到最小值为$-2 \log 2$：
$$P_{G}(x) = P_{data}(x)$$

### 算法：
- $G^{*}=arg \ \underset{G}{\text {min}} \ \underset{D}{\text {max}} \ V(G, D)$，先将$\underset{D}{\text {max}} \ V(G, D)$ 设为$L(G)$，$\underset{D}{\text {max}} \ V(G, D)$只跟$G$有关，给定$G$找到一个$D$的最大值。$L(G)$表示Generator G 的损失函数（训练神经网络通过寻找参数去最小化损失函数）；
- 通过梯度下降找一组参数最小化损失函数$L(G)$，获得最佳$G$，$ \Theta _{G}$表示$G$的参数：
$$\Theta _{G} \leftarrow \Theta _{G} - \eta  \frac {\partial L(G)}{\partial \Theta _{G}}$$

#### 如何对带有最大化的式子进行梯度下降？
假设有一个式子为$f(x) = max{D_{1}(x), D_{2}(x), D_{3}(x)}$，$D_{1}(x)$为一条直线如下图，对$f(x)$求导，看$x$落在哪个区域里，如果在最左边的区域里，则对$D_{1}(x)$进行微分，中间区域，则对$D_{2}(x)$进行微分，看落在哪个区域的值最大，则对那个$D_{i}(x)$进行微分：
$$\frac {df(x)}{dx}=\frac {dD_{i}(x)}{dx}$$
求导过程：随机初始在最左边区域，$D_{1}$值最大，则对$D_{1}$进行微分，即$\frac {dD_{1}(x)}{dx}$。然后更新参数是往右走，这时$D_{2}$值最大，则$\frac {dD_{2}(x)}{dx}$，以此类推。

In [ ]:
Image('gan08')

以$\underset{D}{\text {max}} V(G, D)$为例，
- 给定$G_{0}$；
- 找出在给定$G_{0}$情况下，让$V(G_{0}, D)$最大的$D_{0}^{*}$，即$D_{0}^{*}$=$\underset{D}{\text {max}} V(G_{0}, D)$，$V(G_{0}, D_{0}^{*})$是$P_{data}(x)$和$P_{G_{0}}(x)$的JS diverence（JS差异，高度）；
- 然后进行微分，更新参数，获得新的$G_{1}$。从$G_{0}$更新参数到$G_{1}$，减小JS divergence：
$$\Theta _{G}\leftarrow \Theta _{G} - \eta \frac {\partial V(G, D_{0}^{*})}{\partial \Theta _{G}} $$
- 根据$G_{1}$，找到让$V(G_{1}, D)$最大的$D_{1}^{*}$，即$D_{1}^{*}$=$\underset{D}{\text {max}} V(G_{1}, D)$，计算$V(G_{1}, D_{1}^{*})$，$P_{data}(x)$和$P_{G_{1}}(x)$的JS diverence（JS差异）；
- 微分，更新参数，获得新的$G_{2}$：
$$\Theta _{G}\leftarrow \Theta _{G} - \eta \frac {\partial V(G, D_{1}^{*})}{\partial \Theta _{G}} $$
- ......

#### 实际操作中：
- $V=E_{x\sim P_{data}}[\log D(x)] + E_{x\sim P_{G}}[\log (1 - D(x))]$，实际中是通过抽样来近似期望值$E_{x\sim P_{data}}$和$E_{x\sim P_{G}}$。因为分布的空间为所有可能的image，没法做积分。所以通过抽样方法，从$P_{data}(x)$抽取m个点$(x^{1}, x^{2}, ..., x^{m})$，$generator P_{G}(x)$里抽取m个点$(\widetilde{x}^{1}, \widetilde{x}^{2}, ..., \widetilde{x}^{m})$。换句话说，通过抽样取平均来替代求积分：
$$\text {max} \ \widetilde{V}=\frac{1}{m}\sum_{i=1}^{m} \log D(x^{i}) + \frac{1}{m} \sum_{i=1}^{m} \log (1-D(\widetilde{x}^{i}))$$
- $Discriminator \ D$为二元分类器，参数为$\theta _{d}$。现在从$P_{data}(x)$里去m个点，$(x^{1}, x^{2}, ..., x^{m})$当做正样本。从$P_{G}$(x)里取m个点，$(\widetilde{x}^{1}, \widetilde{x}^{2}, ..., \widetilde{x}^{m})$当做负样本。将最大化积分$V$转为最大化抽样平均$\widetilde{V}$，加上负号，变成最小化损失函数：
$$- \widetilde{V} = L = -\frac{1}{m}\sum_{i=1}^{m} \log D(x^{i}) - \frac{1}{m} \sum_{i=1}^{m} \log (1-D(\widetilde{x}^{i}))$$

总结：训练一个二元分类器，从data中抽样，更新参数最小化损失函数$L$，即可让JS divergence达到最大化。

#### 整个算法流程：
给定一个$Discriminator \ D$和$Generator \ G$，初始化其参数$\theta _{d}$和$\theta {g}$。在每次训练迭代中：
- 从$P_{data}(x)$分布里随机抽取m个样本$(x_{1}, x_{2}, ... , x_{m})$ ；
- 从先验分布$P_{prior}(z)$（如高斯分布、均匀分布等）随机抽取m个样本$(z_{1}, z_{2}, ... , z_{m})$；
- 将先验分布的m个样本$(z_{1}, z_{2}, ... , z_{m})$输入到$Generator \ G$中，输出为$(\widetilde{x}^{1}, \widetilde{x}^{2}, ... , \widetilde{x}^{m})$，即$G(z^{i})=\widetilde{x}^{i}$；
- 在给定$Generator \ G$的情况下，更新$Discriminator \ D$的参数$\theta {d}$以最大化：
$$\widetilde{V} = \frac{1}{m}\sum_{i=1}^{m} \log D(x^{i}) + \frac{1}{m} \sum_{i=1}^{m} \log (1-D(\widetilde{x}^{i}))\\
\theta _{d} \leftarrow \theta _{d} + \eta \bigtriangledown \widetilde{V} (\theta _{d})$$

以上部分为$Discriminator \ D$的训练，通常需要重复K次，以寻找到$JS divergence$的局部最大化值。$Discriminator \ D$的学习是，固定$G$的情况下，让$D$最大。

- 从先验分布$P_{prior}(z)$里随机抽取另外m个样本$(z_{1}, z_{2}, ... , z_{m})$；
- 将m个样本输入到$Generator \ G$里，得到$\widetilde {x}^{i}=G(z^{i})$，固定$D$的情况下，寻找能最小化的$G$：
$$\widetilde{V} = \frac{1}{m}\sum_{i=1}^{m} \log D(x^{i}) + \frac{1}{m} \sum_{i=1}^{m} \log (1-D(G(z^{i}))\\
\theta _{g} \leftarrow \theta _{g} - \eta \bigtriangledown \widetilde{V} (\theta _{g})$$

以上部分为$Generator G$的学习，通常只需更新一次。更新太多，可能没法减少$JS divergence$。

一些技巧方法：
- 在训练$Discriminator \ D$时，输入都加上一些噪声；
- 在$Discriminator \ D$的标签上加噪声，即将真实图片数据集的部分改成$Generator \ G$生成 的图片（假图片）。加了噪声后，有了重叠，这样$Discriminator \ D$没法得到误差为0的结果，可以训练。但噪声需要随时间越来越少，否则会影响$Discriminator \ D$的判断。